## Deliverable 2. Create a Customer Travel Destinations Map.

In [50]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [45]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,78.75,91,20,4.61,thunderstorm
1,1,Ushuaia,AR,-54.8000,-68.3000,40.66,75,40,23.02,scattered clouds
2,2,Conde,BR,-7.2597,-34.9075,76.46,83,0,11.50,clear sky
3,3,Cape Town,ZA,-33.9258,18.4232,55.11,73,40,8.05,scattered clouds
4,4,Gacko,BA,43.1672,18.5353,47.46,95,77,5.53,broken clouds


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,78.75,91,20,4.61,thunderstorm
2,2,Conde,BR,-7.2597,-34.9075,76.46,83,0,11.50,clear sky
5,5,Saint-Joseph,RE,-21.3667,55.6167,68.38,79,22,12.93,light rain
6,6,Gamba,GA,-2.6500,10.0000,74.75,83,100,6.67,overcast clouds
9,9,Zhob,PK,31.3411,69.4481,73.83,51,0,1.52,clear sky
10,10,Hithadhoo,MV,-0.6000,73.0833,81.32,73,11,4.65,few clouds
11,11,Japura,ID,-0.3167,102.3500,72.39,98,99,2.62,overcast clouds
17,17,Kapaa,US,22.0752,-159.3190,84.18,70,82,4.00,broken clouds
18,18,Ribeira Grande,PT,38.5167,-28.7000,74.61,91,100,6.78,overcast clouds
19,19,Kutum,SD,14.2000,24.6667,72.88,67,13,5.23,few clouds


In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
707,False,False,False,False,False,False,False,False,False,False
709,False,False,False,False,False,False,False,False,False,False
710,False,False,False,False,False,False,False,False,False,False
715,False,False,False,False,False,False,False,False,False,False


In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,78.75,thunderstorm,5.4112,100.3354,
2,Conde,BR,76.46,clear sky,-7.2597,-34.9075,
5,Saint-Joseph,RE,68.38,light rain,-21.3667,55.6167,
6,Gamba,GA,74.75,overcast clouds,-2.6500,10.0000,
9,Zhob,PK,73.83,clear sky,31.3411,69.4481,
10,Hithadhoo,MV,81.32,few clouds,-0.6000,73.0833,
11,Japura,ID,72.39,overcast clouds,-0.3167,102.3500,
17,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
18,Ribeira Grande,PT,74.61,overcast clouds,38.5167,-28.7000,
19,Kutum,SD,72.88,few clouds,14.2000,24.6667,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    location = f"{lat}, {lng}"
    
    params['location'] = location
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    results = response['results']
    
    try:
        print(f"The closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        
    print("------------")
        

The closest hotel is Cititel Penang.
------------
The closest hotel is Pousada Beija Flor.
------------
The closest hotel is "Plantation Bed and Breakfast.
------------
Missing field/result... skipping.
------------
The closest hotel is ROOM # 105 DUBAI HOTEL ZHOB.
------------
The closest hotel is Scoop Guest House.
------------
The closest hotel is Rumah mbah Jamah.
------------
The closest hotel is Sheraton Kauai Coconut Beach Resort.
------------
The closest hotel is Quinta da Meia Eira.
------------
The closest hotel is Dibbo areaمنطقة دبو.
------------
The closest hotel is Pereybere Hotel & Spa.
------------
The closest hotel is People ThankYou.
------------
The closest hotel is Pension Oaoa.
------------
The closest hotel is Hotel Castrum Villae by Walk Hotels.
------------
Missing field/result... skipping.
------------
The closest hotel is Paradise Inn.
------------
The closest hotel is Isles Sunset Lodge.
------------
The closest hotel is JVL Inn and Cafeteria.
------------
Th

The closest hotel is Hilton Virginia Beach Oceanfront.
------------
Missing field/result... skipping.
------------
The closest hotel is ALETHEIA Appartement Sinnamary.
------------
The closest hotel is Salinas Park Resort Maçarico AP101.
------------
The closest hotel is Yindu Hotel.
------------
The closest hotel is Apartment above Depot Bar.
------------
The closest hotel is Lords Inn Somnath.
------------
The closest hotel is Résidences du Programme FIDA.
------------
The closest hotel is Grand Copthorne Waterfront.
------------
The closest hotel is Casa Dora, Bubaque.
------------
The closest hotel is Hotel Nusantara I.
------------
The closest hotel is LOTTE HOTEL SEOUL.
------------
The closest hotel is Baybay ni Duardo.
------------
The closest hotel is Hotel Kawabata Shibetsu.
------------
The closest hotel is MsLo Kopong residency.
------------
The closest hotel is Qinhuangdao Peninsula Seasons Hotel And Apartment.
------------
The closest hotel is Hotel Manglares.
-----------

The closest hotel is Sitio Sensation.
------------
The closest hotel is Cricaré Praia Hotel.
------------
The closest hotel is Les Baobabs.
------------
The closest hotel is Kimbe Bay Hotel.
------------
The closest hotel is Agriturismo Tibitone.
------------
The closest hotel is Foyer ADM.
------------
The closest hotel is Wingate By Wyndham Rome.
------------
Missing field/result... skipping.
------------
The closest hotel is Baculin Cove Beach Resort.
------------
The closest hotel is Haajy Guest House 1.
------------
The closest hotel is INN NEW HORIZON.
------------
The closest hotel is Hampton Inn & Suites Morgan City.
------------
The closest hotel is Sri Sai bike Paint,manthani.
------------
The closest hotel is Kimbo Hotel.
------------
The closest hotel is Canopy Tribe.
------------
The closest hotel is Semnan Tourist Hotel.
------------
Missing field/result... skipping.
------------
The closest hotel is Continent Hotel.
------------
The closest hotel is Khách sạn Sài Gòn Bạc

In [52]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,78.75,thunderstorm,5.4112,100.3354,Cititel Penang
2,Conde,BR,76.46,clear sky,-7.2597,-34.9075,Pousada Beija Flor
5,Saint-Joseph,RE,68.38,light rain,-21.3667,55.6167,"""Plantation Bed and Breakfast"
9,Zhob,PK,73.83,clear sky,31.3411,69.4481,ROOM # 105 DUBAI HOTEL ZHOB
10,Hithadhoo,MV,81.32,few clouds,-0.6000,73.0833,Scoop Guest House


In [53]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))